For calculating the number of hexes between two points of a 20 sided sphere, in a map such as

https://sites.google.com/site/2300adgame/known-universe/sol/earth

where each edge of a triangular face is spanned by 7 hexes. Input is based on the sphere being flattened to a 2D page

In [10]:
# Read in relational file
with open("2300GG - Unit List - Sheet13.csv", 'r') as file:
    lines = file.readlines()
    hexid = []; name = []; row = []; colpos = []
    for hex in lines[1:493]:
        vals = hex.strip().split(',')
        hexid.append(int(vals[0])); name.append(vals[1]); row.append(int(vals[4]))
        colpos.append(float(vals[5])); colpos.append(float(vals[6])); colpos.append(float(vals[7])); colpos.append(float(vals[8])); colpos.append(float(vals[9])); colpos.append(float(vals[10]))


row : Y coord position, starting at 0 from the bottom

colpos : Xcoord position of the centre of each hex. Starting at 0 from the left edge. Some hexes have multivalued numbers for xcoord position due to being on the edge of a wedge and an inserted wedge between the other piece. This means that colpos can have upto 6 different values, where colpos1 is the xcoord of the left edge of the hex, colpos6 is the right edge, and colpos2-5 (which only happens for the hexes at the poles) are something inbetween.

Testing and development

In [15]:
#calculate distances
for source in range(480,490):  
    for target in range (70,92):   
#set initial distance to be the simplest case, 0 or traversing a pole
        if source == target:
            distance = 0
        else:
            distance = min(row[source] + row[target], (21 - row[source]) + (21 - row[target]) )
            for colpostypesrc in range(6):
#skip where source xcord does not exist
                if colpos[colpostypesrc + source*6] != -9999 :
                    for colpostypetar in range(6):
#skip where target xcord does not exist
                        if colpos[colpostypetar + target*6] != -9999 : 
#where the closest is wrapped around the left/right edge
                            for wrapsrc in range(2):
                                for wraptar in range(2):
#skip where both source and target wrapped as it is a duplicate of neither
                                    if (wrapsrc + wraptar)!=2:
                                        diffrow = abs(row[source] - row[target])
#shift column depending on being multivalued or wrapping
                                        sourcecol = colpos[colpostypesrc + source*6] + wrapsrc*35
                                        targetcol = colpos[colpostypetar + target*6] + wraptar*35
#skip if source and target are separated by more than 1/2 the map
                                        if (abs(sourcecol - targetcol)) <= 18.5:
#wedgewidth for removal of inserted space when traversing empty wedges, the shortest path is traversing a row closest to a pole
                                            wedgewidthfull = max(max(0, -row[source] + 7), max(0, -row[target] + 7), max(0, row[source] - 14), max(0, row[target] - 14))
#wedgenum for removal of inserted space when traversing empty wedges, counting the number of empty wedges
                                            wedgenum = 0
                                            wedgewidthpartial = 0
                                            if (sourcecol != targetcol) or (wedgewidthfull != 0): 
#assume source on left side, target to the right. Where equal is already handled.
                                                trailingcol = sourcecol; trailingrow =row[source]
                                                leadingcol = targetcol; leadingrow = row[target]

#case of source on right side, target to the left. Where equal is already handled.
                                                if sourcecol>targetcol:
                                                    trailingcol = targetcol; trailingrow = row[target]
                                                    leadingcol = sourcecol; leadingrow = row[source]

#case of path traverses the extremes of the N hemisphere only. 
                                                if (row[source] >=7 and row[target] >=7):
                                                    wedgepostrailing = (trailingcol + 10.5 - trailingrow*0.5)  
                                                    wedgeposleading = (leadingcol - 3.5 + leadingrow*0.5)
#Counting number of empty wedges                                         
                                                    wedgenum = abs(wedgeposleading//7 - wedgepostrailing//7)

#The first, or last, traversed missing wedge will be a partial
                                                    wedgewidthpartial = min(wedgewidthfull, 7 - wedgepostrailing % 7, wedgeposleading % 7)

#adjust for rounding that makes the outer edge of the slanted column be assigned to the slanted column to the right  
                                                    if wedgeposleading % 7 == 0:
                                                        wedgenum = abs((wedgeposleading -7)//7 - wedgepostrailing//7)
                                                        if leadingrow < trailingrow and leadingrow >= 7:
                                                            wedgewidthpartial = wedgewidthfull
#case of path traverses the extremes of the S hemisphere only
                                                elif (row[source] <=14 and row[target] <=14):
                                                    wedgepostrailing = (trailingcol - 3.5 + trailingrow*0.5)  
                                                    wedgeposleading = (leadingcol + 10.5 - leadingrow*0.5)
#Counting number of empty wedges                                         
                                                    wedgenum = abs(wedgeposleading//7 - wedgepostrailing//7) -1

#The first, or last, traversed missing wedge will be a partial
                                                    wedgewidthpartial = min(wedgewidthfull, 7 - wedgepostrailing % 7, wedgeposleading % 7)

#adjust for rounding that makes the outer edge of the slanted column be assigned to the slanted column to the right  
                                                    if wedgeposleading % 7 == 0:
                                                        wedgenum = abs((wedgeposleading -7)//7 - wedgepostrailing//7) -1
                                                        if leadingrow > trailingrow and leadingrow <= 14:
                                                            wedgewidthpartial = wedgewidthfull
#case of path traverses the extremes of the N AND S hemisphere
                                                else:
                                                    wedgepostrailing = (trailingcol + 10.5 - trailingrow*0.5)  
                                                    wedgeposleading = (leadingcol - 3.5 + leadingrow*0.5)
#Counting number of empty wedges                                         
                                                    wedgenum = abs(wedgeposleading//7 - wedgepostrailing//7)

#The first, or last, traversed missing wedge will be a partial
                                                    if wedgenum == 0:
                                                        wedgewidthpartial = 0
                                                    else:
                                                        wedgewidthpartialnorm = min(wedgewidthfull, 7 - wedgepostrailing % 7, wedgeposleading % 7)
                                                        wedgewidthpartialalt = min(max(-trailingrow + 7, -leadingrow + 7), max(trailingrow -14, leadingrow -14))
                                                        wedgewidthpartial = max(wedgewidthpartialalt, wedgewidthpartialnorm)

                        
#adjust for rounding that makes the outer edge of the slanted column be assigned to the slanted column to the right  
                                                    if wedgeposleading % 7 == 0:
                                                        wedgenum = abs((wedgeposleading -7)//7 - wedgepostrailing//7)
                                                        wedgewidthpartial = wedgewidthfull
#case of path traverses the middle
#does not matter as wedgewidth will be 0

#totals shift to source and target column
                                            if wedgenum >= 2:
                                                wedgeshift =  wedgewidthpartial + wedgewidthfull*(wedgenum - 1)
                                            elif wedgenum == 1:
                                                wedgeshift =  wedgewidthpartial
                                            else:
                                                wedgeshift = 0
                                            diffcol = abs(sourcecol - targetcol)  - wedgeshift
                                            if (row[source]  == 2) :
                                                print("B", name[source], name[target], source, target, "colpostype:", colpostypesrc, colpostypetar, "colpos:", colpos[colpostypesrc + source*6], colpos[colpostypetar + target*6], "col:", sourcecol, targetcol, "wrap:", wrapsrc, wraptar,"wedgeshift:", wedgepostrailing, wedgeposleading, wedgewidthfull, wedgewidthpartial, wedgenum, wedgeshift, "diff:", diffrow, diffcol, "distance:", distance)
#diffcol is reduced by diffrow *0.5 to account for diagonal movemement. 
                                            diffcol = max(0, diffcol - diffrow*0.5)
                                            testdistance = diffcol + diffrow
                                            if testdistance < distance:
                                                distance = testdistance
                                            if (row[source]  == 2):
                                                print("C", name[source], name[target], source, target, "colpostype:", colpostypesrc, colpostypetar, "colpos:", colpos[colpostypesrc + source*6], colpos[colpostypetar + target*6], "col:", sourcecol, targetcol, "wrap:", wrapsrc, wraptar,"wedgeshift:", wedgepostrailing, wedgeposleading, wedgewidthfull, wedgewidthpartial, wedgenum, wedgeshift, "diff:", diffrow, diffcol, "distance:", testdistance, distance)


B 9S5 6N20 480 70 colpostype: 0 0 colpos: 11.5 27.5 col: 11.5 27.5 wrap: 0 0 wedgeshift: 21.0 32.0 5 4.0 1.0 4.0 diff: 14 12.0 distance: 18
C 9S5 6N20 480 70 colpostype: 0 0 colpos: 11.5 27.5 col: 11.5 27.5 wrap: 0 0 wedgeshift: 21.0 32.0 5 4.0 1.0 4.0 diff: 14 5.0 distance: 19.0 18
B 9S5 6N20 480 70 colpostype: 0 5 colpos: 11.5 27.5 col: 11.5 27.5 wrap: 0 0 wedgeshift: 21.0 32.0 5 4.0 1.0 4.0 diff: 14 12.0 distance: 18
C 9S5 6N20 480 70 colpostype: 0 5 colpos: 11.5 27.5 col: 11.5 27.5 wrap: 0 0 wedgeshift: 21.0 32.0 5 4.0 1.0 4.0 diff: 14 5.0 distance: 19.0 18
B 9S5 6N20 480 70 colpostype: 5 0 colpos: 16.5 27.5 col: 16.5 27.5 wrap: 0 0 wedgeshift: 26.0 32.0 5 2 1.0 2 diff: 14 9.0 distance: 18
C 9S5 6N20 480 70 colpostype: 5 0 colpos: 16.5 27.5 col: 16.5 27.5 wrap: 0 0 wedgeshift: 26.0 32.0 5 2 1.0 2 diff: 14 2.0 distance: 16.0 16.0
B 9S5 6N20 480 70 colpostype: 5 5 colpos: 16.5 27.5 col: 16.5 27.5 wrap: 0 0 wedgeshift: 26.0 32.0 5 2 1.0 2 diff: 14 9.0 distance: 16.0
C 9S5 6N20 480 70 

Production run

In [11]:
#setup outputfile
filename = "hexdistances.csv"
outfile = open(filename, "w")
outfile.write(" " + ",")
for source in range(0,492): 
    outfile.write(name[source] + ",")
outfile.write("\n")

#calculate distances
for source in range(0,492):  
    outfile.write(name[source] + ",")
    for target in range (0,492):   
#set initial distance to be the simplest case, 0 or traversing a pole
        if source == target:
            distance = 0
        else:
            distance = min(row[source] + row[target], (21 - row[source]) + (21 - row[target]) )
            for colpostypesrc in range(6):
#skip where source xcord does not exist
                if colpos[colpostypesrc + source*6] != -9999 :
                    for colpostypetar in range(6):
#skip where target xcord does not exist
                        if colpos[colpostypetar + target*6] != -9999 : 
#where the closest is wrapped around the left/right edge
                            for wrapsrc in range(2):
                                for wraptar in range(2):
#skip where both source and target wrapped as it is a duplicate of neither
                                    if (wrapsrc + wraptar)!=2:
                                        diffrow = abs(row[source] - row[target])
#shift column depending on being multivalued or wrapping
                                        sourcecol = colpos[colpostypesrc + source*6] + wrapsrc*35
                                        targetcol = colpos[colpostypetar + target*6] + wraptar*35
#skip if source and target are separated by more than 1/2 the map
                                        if (abs(sourcecol - targetcol)) <= 18.5:
#wedgewidth for removal of inserted space when traversing empty wedges, the shortest path is traversing a row closest to a pole
                                            wedgewidthfull = max(max(0, -row[source] + 7), max(0, -row[target] + 7), max(0, row[source] - 14), max(0, row[target] - 14))
#wedgenum for removal of inserted space when traversing empty wedges, counting the number of empty wedges
                                            wedgenum = 0
                                            wedgewidthpartial = 0
                                            if (sourcecol != targetcol) or (wedgewidthfull != 0): 
#assume source on left side, target to the right. Where equal is already handled.
                                                trailingcol = sourcecol; trailingrow =row[source]
                                                leadingcol = targetcol; leadingrow = row[target]

#case of source on right side, target to the left. Where equal is already handled.
                                                if sourcecol>targetcol:
                                                    trailingcol = targetcol; trailingrow = row[target]
                                                    leadingcol = sourcecol; leadingrow = row[source]

#case of path traverses the extremes of the N hemisphere only. 
                                                if (row[source] >=7 and row[target] >=7):
                                                    wedgepostrailing = (trailingcol + 10.5 - trailingrow*0.5)  
                                                    wedgeposleading = (leadingcol - 3.5 + leadingrow*0.5)
#Counting number of empty wedges                                         
                                                    wedgenum = abs(wedgeposleading//7 - wedgepostrailing//7)

#The first, or last, traversed missing wedge will be a partial
                                                    wedgewidthpartial = min(wedgewidthfull, 7 - wedgepostrailing % 7, wedgeposleading % 7)

#adjust for rounding that makes the outer edge of the slanted column be assigned to the slanted column to the right  
                                                    if wedgeposleading % 7 == 0:
                                                        wedgenum = abs((wedgeposleading -7)//7 - wedgepostrailing//7)
                                                        if leadingrow < trailingrow and leadingrow >= 7:
                                                            wedgewidthpartial = wedgewidthfull
#case of path traverses the extremes of the S hemisphere only
                                                elif (row[source] <=14 and row[target] <=14):
                                                    wedgepostrailing = (trailingcol - 3.5 + trailingrow*0.5)  
                                                    wedgeposleading = (leadingcol + 10.5 - leadingrow*0.5)
#Counting number of empty wedges                                         
                                                    wedgenum = abs(wedgeposleading//7 - wedgepostrailing//7) -1

#The first, or last, traversed missing wedge will be a partial
                                                    wedgewidthpartial = min(wedgewidthfull, 7 - wedgepostrailing % 7, wedgeposleading % 7)

#adjust for rounding that makes the outer edge of the slanted column be assigned to the slanted column to the right  
                                                    if wedgeposleading % 7 == 0:
                                                        wedgenum = abs((wedgeposleading -7)//7 - wedgepostrailing//7) -1
                                                        if leadingrow > trailingrow and leadingrow <= 14:
                                                            wedgewidthpartial = wedgewidthfull
#case of path traverses the extremes of the N AND S hemisphere
                                                else:
                                                    wedgepostrailing = (trailingcol + 10.5 - trailingrow*0.5)  
                                                    wedgeposleading = (leadingcol - 3.5 + leadingrow*0.5)
#Counting number of empty wedges                                         
                                                    wedgenum = abs(wedgeposleading//7 - wedgepostrailing//7)

#The first, or last, traversed missing wedge will be a partial
                                                    if wedgenum == 0:
                                                        wedgewidthpartial = 0
                                                    else:
                                                        wedgewidthpartialnorm = min(wedgewidthfull, 7 - wedgepostrailing % 7, wedgeposleading % 7)
                                                        wedgewidthpartialalt = min(max(-trailingrow + 7, -leadingrow + 7), max(trailingrow -14, leadingrow -14))
                                                        wedgewidthpartial = max(wedgewidthpartialalt, wedgewidthpartialnorm)

                        
#adjust for rounding that makes the outer edge of the slanted column be assigned to the slanted column to the right  
                                                    if wedgeposleading % 7 == 0:
                                                        wedgenum = abs((wedgeposleading -7)//7 - wedgepostrailing//7)
                                                        wedgewidthpartial = wedgewidthfull
#case of path traverses the middle
#does not matter as wedgewidth will be 0

#totals shift to source and target column
                                            if wedgenum >= 2:
                                                wedgeshift =  wedgewidthpartial + wedgewidthfull*(wedgenum - 1)
                                            elif wedgenum == 1:
                                                wedgeshift =  wedgewidthpartial
                                            else:
                                                wedgeshift = 0
                                            diffcol = abs(sourcecol - targetcol)  - wedgeshift
#diffcol is reduced by diffrow *0.5 to account for diagonal movemement. 
                                            diffcol = max(0, diffcol - diffrow*0.5)
                                            testdistance = diffcol + diffrow
                                            if testdistance < distance:
                                                distance = testdistance
        outfile.write(str(distance) + ",")
    outfile.write("\n")
outfile.close()